In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys


import functions.parse_data as parse
import functions.handy_functions as hf
import torch.nn as nn
import torch

from multivariate_quantile_regression.network_model import QuantileNetwork

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

from tqdm import tqdm

import os

import random

/tmp/ipykernel_113190/3956026877.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-02-19 14:43:33.654237: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-19 14:43:33.691789: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-19 14:4

In [2]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # CUDA is available, so let's set default device to GPU
    torch.set_default_device(0)
    print("CUDA is available. Using GPU.")
else:
    # CUDA is not available, so let's use the CPU
    print("CUDA is not available. Using CPU.")

# Example usage:
tensor = torch.randn(3, 3)  # Create a tensor on the selected device
print("Tensor is on device:", tensor.device)

CUDA is available. Using GPU.
Tensor is on device: cuda:0


In [3]:
channel_labels= ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06','Cloud_B07',
                 'Cloud_B08','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12','Cloud_B13']

data_water=parse.parse('cloudrm_water.dat')
data_clear=parse.parse('cloudrm_clear.dat')
data_ice=parse.parse('cloudrm_ice.dat')
data_mixed=parse.parse('cloudrm_mixed.dat')

#Concatinate all datasets
data_all=pd.concat([data_water, data_clear, data_ice, data_mixed])
data_all=data_all.drop(columns=['Surface_Desc','Cloud_B01','Clear_B01'])
df_truth=data_all.copy()

data_all=hf.add_MSI_noise(data_all,channel_labels)

Noise standard deviation for Cloud_B02: 0.00335001428051948
Noise standard deviation for Cloud_B03: 0.002912530185416667
Noise standard deviation for Cloud_B04: 0.004058081082042254
Noise standard deviation for Cloud_B05: 0.0046524891611111115
Noise standard deviation for Cloud_B06: 0.007455351321348316
Noise standard deviation for Cloud_B07: 0.008871707484285717
Noise standard deviation for Cloud_B08: 0.04489677938000001
Noise standard deviation for Cloud_B09: 0.005688141120114942
Noise standard deviation for Cloud_B10: 0.003909328971491229
Noise standard deviation for Cloud_B11: 0.0014014724139999996
Noise standard deviation for Cloud_B12: 0.005030040539999999
Noise standard deviation for Cloud_B13: 0.004041267081999999


In [4]:
##Train test validation split##
X_labels= ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06',
           'Cloud_B07','Cloud_B08','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12','Cloud_B13',
           'Sat_Zenith_Angle','Sun_Zenith_Angle','Azimuth_Diff_Angle']

#Leave out 'GOT', 'Water_Vapor'
#Band 1 no go.
y_labels=['Clear_B02','Clear_B03','Clear_B04','Clear_B05','Clear_B06',
           'Clear_B07','Clear_B08','Clear_B09','Clear_B10','Clear_B11','Clear_B12','Clear_B13']

X_truth=df_truth[X_labels]

df=hf.normalise_input_df(data_all,X_labels)
df=hf.add_noise(df,X_labels,sigma=0.001)

##Split data##
X=df[X_labels]
y=df[y_labels]

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.05)

X_test_truth=X_truth.iloc[X_test.index] #Save truth values without normalisation

In [6]:
sequence= lambda: nn.Sequential(
    nn.Linear(len(X_labels),256),
    nn.ReLU(),
    nn.Linear(256,256),
    nn.ReLU(),
    nn.Linear(256, len(quantiles)*len(y_labels)) #Output dimesion is number of quantiles times number of target variables
)

In [20]:
quantiles=np.array([0.1,0.5,0.9])
num_epochs=[10,20]
num_models=3 #Set number of ensambles
batch_size=500
nepochs=200
lr=0.001
val_size=0.05
data_norm=False

looped_metric="Loop_nepochs"
main_filepath='pytorch_models/'+looped_metric

model_metrics=pd.DataFrame(columns=['Ensemble_mean','Ensemble_index',looped_metric,'MSE','PSNR','R2_score','Mean_Quantile_Loss','Quant_Rate','Quantiles'])
for nepochs in num_epochs:

    
    models=[QuantileNetwork(quantiles=quantiles) for _ in range(num_models)]
    preds_total=[]
    for i,model in enumerate(models):

        validation_indices=np.array(random.sample(range(len(X_train['Cloud_B02'])), int(len(X_train['Cloud_B02'])*val_size)))
        train_indices=[i for i in range(len(X_train['Cloud_B02'])) if np.any(validation_indices==i)==False]

        model.fit(X_train.to_numpy(),y_train.to_numpy(), 
            train_indices, 
            validation_indices, 
            batch_size=batch_size,
            nepochs=nepochs,
            sequence=sequence(),
            lr=lr,
            data_norm=data_norm)
        
        filepath='pytorch_models/'+looped_metric+'/'+str(nepochs)+'/model'+str(i)
        os.makedirs(filepath,exist_ok=True)
        torch.save(model,filepath+'/model_file')

        preds = model.predict(X_test.to_numpy())
        if i==0:
            preds_total=preds
        else:
            preds_total=preds_total+preds

        mse=mean_squared_error(y_test.to_numpy(),preds[:,:,1])
        psnr=QuantileNetwork.PSNR(y_test,preds[:,:,1])
        r2=r2_score(y_test.to_numpy(),preds[:,:,1])
        mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds,quantiles)
        quant_rate=QuantileNetwork.quant_rate(y_test.to_numpy(),preds)

        tmp_metrics=pd.DataFrame(data=[[False,i,nepochs,mse,psnr,r2,mean_quantile,quant_rate,quantiles]],columns=['Ensemble_mean','Ensemble_index',looped_metric,'MSE','PSNR','R2_score','Mean_Quantile_Loss','Quant_Rate','Quantiles'])
        model_metrics=pd.concat([model_metrics,tmp_metrics])
    
    preds_total=preds_total/num_models

    mse=mean_squared_error(y_test.to_numpy(),preds_total[:,:,1])
    psnr=QuantileNetwork.PSNR(y_test,preds_total[:,:,1])
    r2=r2_score(y_test.to_numpy(),preds_total[:,:,1])
    mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds_total,quantiles)
    quant_rate=QuantileNetwork.quant_rate(y_test.to_numpy(),preds_total)

    tmp_metrics=pd.DataFrame(data=[[True,np.nan,nepochs,mse,psnr,r2,mean_quantile,quant_rate,quantiles]],columns=['Ensemble_mean','Ensemble_index',looped_metric,'MSE','PSNR','R2_score','Mean_Quantile_Loss','Quant_Rate','Quantiles'])
    model_metrics=pd.concat([model_metrics,tmp_metrics])


X_test.to_csv(main_filepath+'/xtest.csv',index=False)
y_test.to_csv(main_filepath+'/ytest.csv',index=False)

model_metrics=model_metrics.reset_index(drop=True)
model_metrics.to_csv(main_filepath+'/model_metrics.csv',index=False)

Epoch 1


Batch number:   0%|          | 0/361 [00:00<?, ?it/s]

Batch number: 100%|██████████| 361/361 [00:00<00:00, 371.33it/s]

Training loss [0.91167796] Validation loss [0.64027846]
----New best validation loss---- [0.64027846]
Epoch 2



Batch number: 100%|██████████| 361/361 [00:00<00:00, 394.20it/s]

...


Batch number: 100%|██████████| 361/361 [00:00<00:00, 496.11it/s]

Training loss [0.40287885] Validation loss [0.39737186]
Epoch 20



Batch number: 100%|██████████| 361/361 [00:00<00:00, 506.54it/s]

Training loss [0.40044883] Validation loss [0.39219218]


----New best validation loss---- [0.39219218]
Best model out of total max epochs found at epoch 20


In [22]:
model_metrics

,Ensemble_mean,Ensemble_index,Loop_nepochs,MSE,PSNR,R2_score,Mean_Quantile_Loss,Quant_Rate,Quantiles
0,False,0,10,0.004986,27.704115,0.932560,0.436629,"[0.12181666666666667, 0.6030833333333333, 0.94...","[0.1, 0.5, 0.9]"
1,False,1,10,0.004992,27.698792,0.928910,0.428881,"[0.081875, 0.5132166666666667, 0.9187083333333...","[0.1, 0.5, 0.9]"
2,False,2,10,0.005005,27.687414,0.930575,0.432807,"[0.07501666666666666, 0.44104166666666667, 0.8...","[0.1, 0.5, 0.9]"
3,True,NaN,10,0.004782,27.884982,0.933763,0.412212,"[0.07074166666666666, 0.524575, 0.941525]","[0.1, 0.5, 0.9]"
4,False,0,20,0.004663,27.994436,0.935145,0.406918,"[0.12814166666666665, 0.6223166666666666, 0.94...","[0.1, 0.5, 0.9]"
5,False,1,20,0.004528,28.122021,0.936586,0.400809,"[0.079125, 0.46773333333333333, 0.906733333333...","[0.1, 0.5, 0.9]"
6,False,2,20,0.004463,28.184926,0.937791,0.399300,"[0.12109166666666667, 0.5491666666666667, 0.91...","[0.1, 0.5, 0.9]"
7,True,NaN,20,0.004335,28.310952,0.939406,0.381722,"[0.07736666666666667, 0.5702833333333334, 0.94...","[0.1, 0.5, 0.9]"
